# 如何使用hyperopt对Lightgbm进行自动调参

之前的教程以及介绍过如何使用hyperopt对xgboost进行调参，并且已经说明了，该代码模板可以十分轻松的转移到lightgbm，或者catboost上。而本篇教程就是对原模板的一次歉意，前半部分为[教程-如何使用hyperopt对xgboost进行自动调参](https://zhuanlan.zhihu.com/p/52562126)的迁移，后半部分是对[在Hyperopt框架下使用XGboost与交叉验证](https://zhuanlan.zhihu.com/p/52562732)的迁移。


## 关于Hyperopt

Hyperopt：是python中的一个用于"分布式异步算法组态/超参数优化"的类库。使用它我们可以拜托繁杂的超参数优化过程，自动获取最佳的超参数。广泛意义上，可以将带有超参数的模型看作是一个必然的非凸函数，因此hyperopt几乎可以稳定的获取比手工更加合理的调参结果。尤其对于调参比较复杂的模型而言，其更是能以远快于人工调参的速度同样获得远远超过人工调参的最终性能。

目前中文文档的地址由本人FontTian在2017年翻译，但是hyperopt文档本身确实写的不怎么样。所以才有了这份教程。源代码请前往[Github教程地址下载](https://github.com/FontTian/hyperopt-doc-zh/tree/master/tutorials/zh)下载。

 - [中文文档地址](https://github.com/FontTian/hyperopt-doc-zh)
 - [FontTian的博客](https://blog.csdn.net/fontthrone)
 - [Hyperopt官方文档地址](http://hyperopt.github.io/)


# 迁移一：自动调参

## 获取数据
这里我们使用UCI的红酒质量数据集，除此之外我还额外增加了两个特征。

In [1]:
import numpy as np
import pandas as pd

def GetNewDataByPandas():
    wine = pd.read_csv("/home/fonttian/Data/UCI/wine/wine.csv")
    wine['alcohol**2'] = pow(wine["alcohol"], 2)
    wine['volatileAcidity*alcohol'] = wine["alcohol"] * wine['volatile acidity']
    y = np.array(wine.quality)
    X = np.array(wine.drop("quality", axis=1))

    columns = np.array(wine.columns)

    return X, y, columns

## 分割数据并转换
首先将数据分割为三份，一部分用于预测，训练数据则同样分成额外的两部分用于evallist参数。

同时为了加快速度和减少内存，我们将数据转换为`xgboost`自带的读取格式。

In [2]:
from sklearn.model_selection import train_test_split
# Read wine quality data from file
X, y, wineNames = GetNewDataByPandas()

# split data to [[0.8,0.2],01]
x_train_all, x_predict, y_train_all, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)

x_train, x_test, y_train, y_test = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=100)

import lightgbm as lgb

train_data = lgb.Dataset(data=x_train,label=y_train)
test_data = lgb.Dataset(data=x_test,label=y_test)

## 定义参数空间

使用hyperopt自带的函数定义参数空间，但是因为其`randint()`方法产生的数组范围是从0开始的，所以我额外定义了一个数据转换方法，对原始参数空间进行一次转换。

关于hyperopt中定义参数区间需要使用的函数请参考：
 - 中文地址，请点击[这里](https://github.com/FontTian/hyperopt-doc-zh/wiki/FMin)
 - 英文地址，请点击[这里](https://github.com/FontTian/hyperopt-doc-zh/blob/master/hyperopt/doc/en/FMin.md)

In [4]:
from hyperopt import fmin, tpe, hp, partial

# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 15),
         "num_trees": hp.randint("num_trees", 300),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "bagging_fraction": hp.randint("bagging_fraction", 5),
         "num_leaves": hp.randint("num_leaves", 6),
         }

def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['num_trees'] = argsDict['num_trees'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["bagging_fraction"] = argsDict["bagging_fraction"] * 0.1 + 0.5
    argsDict["num_leaves"] = argsDict["num_leaves"] * 3 + 10
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

## 创建模型工厂与分数获取器

lightgbm模型工厂用于生产我们需要的model，而分数获取器则是为了解耦。这样在实际的测试工作中更加套用代码和修改。

In [14]:
from sklearn.metrics import mean_squared_error

def lightgbm_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)
    
    params = {'nthread': -1,  # 进程数
              'max_depth': argsDict['max_depth'],  # 最大深度
              'num_trees': argsDict['num_trees'],  # 树的数量
              'eta': argsDict['learning_rate'],  # 学习率
              'bagging_fraction': argsDict['bagging_fraction'],  # 采样数
              'num_leaves': argsDict['num_leaves'],  # 终点节点最小样本占比的和
              'objective': 'regression',
              'feature_fraction': 0.7,  # 样本列采样
              'lambda_l1': 0,  # L1 正则化
              'lambda_l2': 0,  # L2 正则化
              'bagging_seed': 100,  # 随机种子,light中默认为100
              }
    params['metric'] = ['rmse']

    model_lgb = lgb.train(params, train_data, num_boost_round=300, valid_sets=[test_data],early_stopping_rounds=100)

    return get_tranformer_score(model_lgb)

def get_tranformer_score(tranformer):
    
    model = tranformer
    prediction = model.predict(x_predict, num_iteration=model.best_iteration)
  
    return mean_squared_error(y_predict, prediction)

### 调用Hyperopt开始调参
之后我们调用hyperopt进行自动调参即可，同时通过返回值获取最佳模型的结果。

In [17]:
# 开始使用hyperopt进行自动调参
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(lightgbm_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)

/home/fonttian/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 0.793788
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 0.776669
[3]	valid_0's rmse: 0.762522
[4]	valid_0's rmse: 0.749776
[5]	valid_0's rmse: 0.738227
[6]	valid_0's rmse: 0.726538
[7]	valid_0's rmse: 0.714767
[8]	valid_0's rmse: 0.704253
[9]	valid_0's rmse: 0.69547
[10]	valid_0's rmse: 0.686395
[11]	valid_0's rmse: 0.678254
[12]	valid_0's rmse: 0.671785
[13]	valid_0's rmse: 0.664915
[14]	valid_0's rmse: 0.658876
[15]	valid_0's rmse: 0.654077
[16]	valid_0's rmse: 0.649826
[17]	valid_0's rmse: 0.646188
[18]	valid_0's rmse: 0.641598
[19]	valid_0's rmse: 0.638169
[20]	valid_0's rmse: 0.634488
[21]	valid_0's rmse: 0.631252
[22]	valid_0's rmse: 0.628054
[23]	valid_0's rmse: 0.625935
[24]	valid_0's rmse: 0.623471
[25]	valid_0's rmse: 0.622281
[26]	valid_0's rmse: 0.619744
[27]	valid_0's rmse: 0.617677
[28]	valid_0's rmse: 0.615377
[29]	valid_0's rmse: 0.61336
[30]	valid_0's rmse: 0.61183
[31]	valid_0's rmse: 0.61012
[32]	valid_0's rmse:

/home/fonttian/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[106]	valid_0's rmse: 0.575037
[107]	valid_0's rmse: 0.573896
[108]	valid_0's rmse: 0.573501
[109]	valid_0's rmse: 0.573449
[110]	valid_0's rmse: 0.573383
[111]	valid_0's rmse: 0.572738
[112]	valid_0's rmse: 0.572494
[113]	valid_0's rmse: 0.571852
[114]	valid_0's rmse: 0.571985
[115]	valid_0's rmse: 0.571215
[116]	valid_0's rmse: 0.571336
[117]	valid_0's rmse: 0.571382
[118]	valid_0's rmse: 0.571758
[119]	valid_0's rmse: 0.571501
[120]	valid_0's rmse: 0.570985
[121]	valid_0's rmse: 0.570496
[122]	valid_0's rmse: 0.570389
[123]	valid_0's rmse: 0.570586
[124]	valid_0's rmse: 0.570719
[125]	valid_0's rmse: 0.570513
[126]	valid_0's rmse: 0.570317
[127]	valid_0's rmse: 0.569815
[128]	valid_0's rmse: 0.569478
[129]	valid_0's rmse: 0.5691
[130]	valid_0's rmse: 0.569457
[131]	valid_0's rmse: 0.568876
[132]	valid_0's rmse: 0.569128
[133]	valid_0's rmse: 0.569325
[134]	valid_0's rmse: 0.569764
[135]	valid_0's rmse: 0.56981
[136]	valid_0's rmse: 0.569497
[137]	valid_0's rmse: 0.569249
[138]	vali

/home/fonttian/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/fonttian/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

[215]	valid_0's rmse: 0.584164
[216]	valid_0's rmse: 0.5841
[217]	valid_0's rmse: 0.583922
[218]	valid_0's rmse: 0.584095
[219]	valid_0's rmse: 0.583845
[220]	valid_0's rmse: 0.58356
[221]	valid_0's rmse: 0.583663
[222]	valid_0's rmse: 0.583467
[223]	valid_0's rmse: 0.583445
[224]	valid_0's rmse: 0.583237
[225]	valid_0's rmse: 0.582877
[226]	valid_0's rmse: 0.582812
[227]	valid_0's rmse: 0.582564
Did not meet early stopping. Best iteration is:
[227]	valid_0's rmse: 0.582564
[1]	valid_0's rmse: 0.792748
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 0.774024
[3]	valid_0's rmse: 0.759242
[4]	valid_0's rmse: 0.746483
[5]	valid_0's rmse: 0.733747
[6]	valid_0's rmse: 0.720609
[7]	valid_0's rmse: 0.709415
[8]	valid_0's rmse: 0.698296
[9]	valid_0's rmse: 0.688935
[10]	valid_0's rmse: 0.679103
[11]	valid_0's rmse: 0.671004
[12]	valid_0's rmse: 0.663518
[13]	valid_0's rmse: 0.656241
[14]	valid_0's rmse: 0.650132
[15]	valid_0's rmse: 0.644707
[16]	valid_0's rm

/home/fonttian/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[200]	valid_0's rmse: 0.565622
[201]	valid_0's rmse: 0.56594
[202]	valid_0's rmse: 0.565723
[203]	valid_0's rmse: 0.565832
[204]	valid_0's rmse: 0.564961
[205]	valid_0's rmse: 0.564795
[206]	valid_0's rmse: 0.564398
[207]	valid_0's rmse: 0.564062
[208]	valid_0's rmse: 0.56386
[209]	valid_0's rmse: 0.563671
[210]	valid_0's rmse: 0.563764
[211]	valid_0's rmse: 0.563605
[212]	valid_0's rmse: 0.563244
[213]	valid_0's rmse: 0.563099
[214]	valid_0's rmse: 0.562782
[215]	valid_0's rmse: 0.562708
[216]	valid_0's rmse: 0.56275
[217]	valid_0's rmse: 0.563106
[218]	valid_0's rmse: 0.563051
[219]	valid_0's rmse: 0.562908
[220]	valid_0's rmse: 0.562789
[221]	valid_0's rmse: 0.56294
[222]	valid_0's rmse: 0.56287
[223]	valid_0's rmse: 0.562789
[224]	valid_0's rmse: 0.562901
[225]	valid_0's rmse: 0.562835
[226]	valid_0's rmse: 0.562643
[227]	valid_0's rmse: 0.562508
[228]	valid_0's rmse: 0.562075
[229]	valid_0's rmse: 0.562136
[230]	valid_0's rmse: 0.562137
[231]	valid_0's rmse: 0.562046
[232]	valid_

/home/fonttian/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[46]	valid_0's rmse: 0.591315
[47]	valid_0's rmse: 0.590454
[48]	valid_0's rmse: 0.589678
[49]	valid_0's rmse: 0.58916
[50]	valid_0's rmse: 0.589106
[51]	valid_0's rmse: 0.58853
[52]	valid_0's rmse: 0.588169
[53]	valid_0's rmse: 0.587408
[54]	valid_0's rmse: 0.587391
[55]	valid_0's rmse: 0.586577
[56]	valid_0's rmse: 0.58626
[57]	valid_0's rmse: 0.585135
[58]	valid_0's rmse: 0.584159
[59]	valid_0's rmse: 0.58374
[60]	valid_0's rmse: 0.583755
[61]	valid_0's rmse: 0.583012
[62]	valid_0's rmse: 0.582222
[63]	valid_0's rmse: 0.582401
[64]	valid_0's rmse: 0.581246
[65]	valid_0's rmse: 0.581349
[66]	valid_0's rmse: 0.581174
[67]	valid_0's rmse: 0.580496
[68]	valid_0's rmse: 0.580108
[69]	valid_0's rmse: 0.580075
[70]	valid_0's rmse: 0.579813
[71]	valid_0's rmse: 0.579797
[72]	valid_0's rmse: 0.579925
[73]	valid_0's rmse: 0.579179
[74]	valid_0's rmse: 0.578663
[75]	valid_0's rmse: 0.578608
[76]	valid_0's rmse: 0.578906
[77]	valid_0's rmse: 0.579065
[78]	valid_0's rmse: 0.578599
[79]	valid_0's

## 展示结果
展示我们获取的最佳参数，以及该模型在训练集上的最终表现，如果想要使用交叉验证请参考其他教程。

In [19]:
RMSE = lightgbm_factory(best)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('rmse of the best lightgbm:', np.sqrt(RMSE))

/home/fonttian/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's rmse: 0.793948
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 0.776625
[3]	valid_0's rmse: 0.76272
[4]	valid_0's rmse: 0.749693
[5]	valid_0's rmse: 0.737787
[6]	valid_0's rmse: 0.725898
[7]	valid_0's rmse: 0.715028
[8]	valid_0's rmse: 0.70466
[9]	valid_0's rmse: 0.695602
[10]	valid_0's rmse: 0.685584
[11]	valid_0's rmse: 0.676378
[12]	valid_0's rmse: 0.669661
[13]	valid_0's rmse: 0.66272
[14]	valid_0's rmse: 0.656383
[15]	valid_0's rmse: 0.650346
[16]	valid_0's rmse: 0.644776
[17]	valid_0's rmse: 0.639606
[18]	valid_0's rmse: 0.635642
[19]	valid_0's rmse: 0.631222
[20]	valid_0's rmse: 0.627905
[21]	valid_0's rmse: 0.624249
[22]	valid_0's rmse: 0.620832
[23]	valid_0's rmse: 0.617602
[24]	valid_0's rmse: 0.616664
[25]	valid_0's rmse: 0.613814
[26]	valid_0's rmse: 0.611469
[27]	valid_0's rmse: 0.609821
[28]	valid_0's rmse: 0.608023
[29]	valid_0's rmse: 0.605641
[30]	valid_0's rmse: 0.603928
[31]	valid_0's rmse: 0.602377
[32]	valid_0's rmse

# 迁移二-交叉验证
## 说明

其实本部分与之前的一样重点依旧在于Hyperopt与Lightgbm参数传入上的一个冲突，不过解决方案很简单，先构建带有Hyperopt参数空间的模型，然后再从构建的模型中获取参数。其实这点xgboost，hyperopt，catboost三个模型的解决方案都一样。catboost自带的教程中也有这种解决方案。只不过catboost自带的教程不和lightgbm与xgboost一样在自己的原项目里，而是在原账号下又额外开了个Github项目，导致不太容易发现。实际上我也是最近在写这个的时候，才发现catboost原来是自带教程的。也正因为如此，本系列教程就不再往catboost上迁移代码了。请自行参考[catboost自带的教程](https://github.com/catboost/tutorials)

## 使用CV方法进行交叉验证

sklearn部分过于简单，本处将不再做迁移。如有需要请自行迁移。

In [47]:
import hyperopt

train_all_data = lgb.Dataset(data=x_train_all,label=y_train_all)

def hyperopt_objective(params):
    
    model = lgb.LGBMRegressor(
        num_leaves=31,
        max_depth=int(params['max_depth']) + 5,
        learning_rate=params['learning_rate'],
        objective='regression',
        eval_metric='rmse',
        nthread=-1,
    )
     
    num_round = 10
    res = lgb.cv(model.get_params(),train_all_data, num_round, nfold=5, metrics='rmse',early_stopping_rounds=10)
    
    return min(res['rmse-mean']) # as hyperopt minimises

In [49]:
# 这里的warnings实在太多了，我们加入代码不再让其显示
import warnings
warnings.filterwarnings("ignore")

from numpy.random import RandomState

params_space = {
    'max_depth': hyperopt.hp.randint('max_depth', 6),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=RandomState(123)
)

print("\n展示hyperopt获取的最佳结果，但是要注意的是我们对hyperopt最初的取值范围做过一次转换")
print(best)


展示hyperopt获取的最佳结果，但是要注意的是我们对hyperopt最初的取值范围做过一次转换
{'learning_rate': 0.059356676015000595, 'max_depth': 5}


## 额外的：主站迁移说明

本文作者FontTian，从2018年12月18日起，技术博客主站将由CSDN转到知乎，同时也会做内容分级：主要内容将发布在知乎，CSDN会同步迁移。CSDN会有的而知乎没有的：一些比较简单的内容，或者草稿。知乎主要发重点内容，以质量为主。而文章链接将默认为知乎链接，部分CSDN专有的除外。

 - [CSDN博客地址](https://blog.csdn.net/FontThrone)
 - [知乎主页](https://www.zhihu.com/people/fonttian/posts)